https://www.metacritic.com/game 에서 크롤링 예정

In [2]:
import re
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup

헤더부분 유저에이전트,레퍼러 예제

In [2]:
hur={'User-Agent':'Mozilla','referer':'https://www.metacritic.com/game/playstation-5/elden-ring'}
url='https://www.metacritic.com/game/playstation-5/elden-ring'
req=requests.get(url,headers=hur)
bsoup=BeautifulSoup(req.content,'html.parser',from_encoding='utf-8') #beautifulsoup객체선언
q1=bsoup.find("div",{"class":"details main_details"})
temp=bsoup.find("span",{"class":"blurb blurb_expanded"}).text
print(temp)

A New World Created By Hidetaka Miyazaki And George R. R. Martin. ELDEN RING, developed by FromSoftware, Inc. and BANDAI NAMCO Entertainment Inc., is a fantasy action-RPG adventure set within a world created by Hidetaka Miyazaki creator of the influential DARK SOULS video game series; and George R.R. Martin author of The New York Times best-selling fantasy series, A Song of Ice and Fire. Danger and discovery lurk around every corner in FromSoftware's largest game to-date.  Hidetaka Miyazaki, President and Game Director of FromSoftware Inc. Known for directing critically-acclaimed games in beloved franchises including Armored Core, Dark Souls, and Sekiro: Shadows Die Twice.  George R.R. Martin is the #1 New York Times bestselling author of many novels, including the acclaimed series A Song of Ice and Fire - A Game of Thrones, A Clash of Kings, A Storm of Swords, A Feast For Crows, and A Dance with Dragons. As a writer-producer, he has worked on The Twilight Zone, Beauty and the Beast, a

이전까지 크롤링했던 진도부터 하기

정해진 플랫폼에 따라 크롤링하는 함수

In [ ]:
def crawl_page(url):
    ret={}
    chk=True
    print(url)
    while(chk):
        req=requests.get(url,headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36','referer':url}) 
        if(req==None): 
            print("reqNone")
            continue
        bsoup=BeautifulSoup(req.text,'lxml')
        time.sleep(1.5)#크롤링 속도 제한
        if(bsoup==None):
            print("bsoupNone")
            continue
        bmetascore=bsoup.find("span",{"itemprop":"ratingValue"})
        if(bmetascore==None):
            time.sleep(4)
            print(bsoup)
            print("bemtaNone")
            continue
        bsummary=bsoup.find("span",class_="blurb blurb_expanded")
        if(bsummary==None):
            print("bsumm")
            bsummary="None"
        else:
            if(bsummary.string==None):
                bsummary=bsoup.find("span",class_="blurb blurb_collapsed")
                if(bsummary.string!=None):
                    bsummary=bsummary.string
                if(bsummary.string!=None):
                    bsummary=bsummary.string
            else:
                bsummary=bsummary.string
            print(bsummary)
        btitle=bsoup.find("div",{"class":"product_title"}).find("h1")
        if(btitle==None): 
            print("btitleNone")
            continue
        buserscore=bsoup.find("div",class_=re.compile("metascore_w user large game"))
        if(buserscore==None): 
            print("buserNone")
            continue
        bpub=bsoup.find("li",class_="summary_detail publisher").find("a")
        if(bpub==None):
            print("bpubNone")
            continue
        bdev=(bsoup.find("li",class_="summary_detail developer"))
        if(bdev==None):
            print("bdevNone")
            continue
        else:
            bdev=bdev.find("a")
        bgenres=(bsoup.find("li",class_="summary_detail product_genre"))
        if(bgenres==None):
            print("bgenreNone")
            continue
        bgenres=bgenres.find_all("span",class_="data")
        sgenres=""
        for x in range(len(bgenres)):
            sgenres=sgenres+str(bgenres[x].string)+"|"
        bgrating=bsoup.find("li",class_="summary_detail product_rating")
        if(bgrating==None):
            print("bgratingNone")
            bgrating="None"
        else:
            bgrating=bgrating.find("span",class_="data").string
        brelease=bsoup.find("li",class_="summary_detail release_data")
        brelease=brelease.find("span",class_="data")
        print(str(btitle.string)," ",str(brelease.string)," ",str(bmetascore.string)," ",str(buserscore.string)," ",str(bpub.string)[57:-53]," ",str(bdev.string)," ",sgenres," ",bgrating)
        ret={"title":str(btitle.string),"metascore":str(bmetascore.string),"userscore":str(buserscore.string),"publisher":str(bpub.string)[57:-53],"developer":str(bdev.string),"genres":sgenres,"rating":str(bgrating),"summary":str(bsummary),"release_date":str(brelease.string)}
        chk=False
    return ret
    
def get_page(url):
    req=requests.get(url,headers={'User-Agent':'Mozilla','referer':url})  
    bsoup=BeautifulSoup(req.content,'html.parser',from_encoding='utf-8')
    bsoup2=bsoup.find_all("td",{"class":"clamp-summary-wrap"})
    lpd=pd.DataFrame(columns={"title","release_date","publisher","developer","genres","rating","summary","metascore","userscore"})
    cnt=0
    for x in bsoup2:
        y=x.find("a")["href"]
        y=y[:-15]
        df_dic=pd.DataFrame([crawl_page("https://www.metacritic.com/"+y)])
        lpd = pd.concat([lpd, df_dic], ignore_index=True)
    return lpd
        
def crawl_direct(platform,df):
    url="https://www.metacritic.com/browse/games/release-date/available/"+platform+"/metascore?page="
    req=requests.get(url,headers={'User-Agent':'Mozilla','referer':url+'0'})
    bsoup=BeautifulSoup(req.content,'html.parser',from_encoding='utf-8')
    lpage_soup=bsoup.find("li",{"class":"last_page"})
    lpage2_soup=lpage_soup.find("a",{"class":"page_num"})
    lpagenum=lpage2_soup.text
    try:
      lpd=pd.read_csv(platform+'.csv')
    except:
        lpd=pd.DataFrame(columns={"title","release_date","publisher","developer","genres","rating","summary","metascore","userscore"})
    proc=int(len(lpd)/100)
    for x in range(proc,int(lpagenum)):  
        print(url+str(x))
        page=get_page(url+str(x))
        page=pd.concat([page,lpd],ignore_index=True)
        page.to_csv(platform+'.csv',sep=',',na_rep='NaN',float_format = '%.2f',index = False)
crawl_direct('ps4',1)

https://www.metacritic.com/browse/games/release-date/available/ps4/metascore?page=2
https://www.metacritic.com//game/playstation-4/guilty-gear--strive-
“Guilty Gear -Strive-“ is the latest entry in the Guilty Gear fighting game franchise. Created by Daisuke Ishiwatari and developed by Arc System Works, “Guilty Gear -Strive-“ upholds the series’ reputation for groundbreaking hybrid 2D/3D cell-shaded graphics and intense, rewarding gameplay.
bgratingNone
Guilty Gear -Strive-   Jun  8, 2021   84   8.1   Arc System Works   Arc System Works   Action|Fighting|2D|   None
https://www.metacritic.com//game/playstation-4/thumper
Thumper strips rhythm gaming down to its core.  Classic rhythm-action is combined with speed and satisfying physicality.  With one analog stick and one button, you control a space beetle while hurtling through stunning and treacherous worlds.  Each rhythmic hit, crash, and impact is interwoven with an original soundtrack by Brian Gibson of the acclaimed noise-rock band Li

Lara Croft Go is a cross between a puzzle game and turn based strategy and for what it is it's close to perfection. The puzzles are really
Lara Croft GO   Dec  3, 2016   84   6.0   Square Enix   Square Enix   Strategy|Turn-Based|General|   E10+
https://www.metacritic.com//game/playstation-4/oddworld-abes-oddysee---new-n-tasty
Rescue your fellow meat factory slaves in this nostalgic, classic platformer with the latest immersive graphics and design. Find your destiny. Once a happy floor waxer in Oddworld's biggest meat processing plant, Abe stumbled across his boss's secret plan to turn the factory's slave labour force into the latest in the RuptureFarms Tasty Treats line of novelty meat snacks. Abe now has to save his own skin from the grinders even if simply escaping the flesh farm is only the start of his Oddysee. Trigger-happy hunting parties, carnivorous sacred beasts, ancient temple traps and even a return to his former workplace await Abe on his journey to discover his people's he

Black Ops returns - in your face. Featuring gritty, grounded, fluid Multiplayer combat, a massive Zombies offering with 3 full undead adventures at launch, and Blackout, where the universe of Black Ops comes to life in one huge battle royale experience featuring the largest map in Call of Duty history, signature Black Ops combat, characters, locations and weapons from the entire Black Ops series. Soldier up for all-out combat – tailor made for the Black Ops community. [Treyarch]
Call of Duty: Black Ops 4   Oct 12, 2018   83   4.0   Activision   Treyarch   Action|Shooter|First-Person|Tactical|   M
https://www.metacritic.com//game/playstation-4/ghost-of-tsushima
The year is 1274. Samurai warriors are the legendary defenders of Japan -- until the fearsome Mongol Empire invades the island of Tsushima, wreaking havoc and conquering the local population. As one of the last surviving samurai, you rise from the ashes to fight back. But, honorable tactics won't lead you to victory. You must mov

Call of Duty: Modern Warfare is back, remastered in true high-definition, featuring improved textures, physically based rendering, high-dynamic range lighting and much more. Relive one of the most iconic campaigns in history, as you are transported around the globe, including fan favorite missions "All Ghillied Up," "Mile High Club," and "Crew Expendable." You'll suit up as unforgettable characters Sgt. John "Soap" MacTavish, Capt. John Price and more, as you battle a rogue enemy group across global hot spots from Eastern Europe and rural Russia, all the way to the Middle East. Through an engaging narrative full of twists and turns, call on sophisticated technology and superior firepower as you coordinate land and air strikes on a battlefield where speed and accuracy are essential to victory.  Additionally, team up with your friends in the online mode that redefined Call of Duty by introducing kill streaks, XP, Prestige and more in customizable, classic multiplayer modes.
Call of Duty:

LEGO Marvel Super Heroes features an original story where players take control of Iron Man, Spider-Man, the Hulk, Captain America, Wolverine and many other Marvel characters as they unite to stop Loki and a myriad of other Marvel villains from assembling a super-weapon capable of destroying the world.
LEGO Marvel Super Heroes   Nov 15, 2013   83   7.7   Warner Bros. Interactive Entertainment   TT Games   Action Adventure|Fantasy|General|Fantasy|Open-World|   E10+
https://www.metacritic.com//game/playstation-4/no-mans-sky-beyond
I’m so blown with how far this game has come. When the game was first announced I was so hyped for it. It’s initial release was sad and it left a bad taste in my mouth for years. After many updates I decided to give it one more try and I am happy to say this is one of my favorite games. Hats off to hello games and their hard work in continually improving the game. Day one no mans sky and 2020 no mans sky are completely different games. Endless amounts of things 

Whether playing alone or online, your mission to build an NFL dynasty begins with Connected Franchise. Interact and build your team with more control with Game Prep. Want to develop your backup QB? Between matchups, spend extra time on him each week to earn training points without having to force him into the real games before he is ready. Build your teams' confidence through performance and front office transactions. Balance keeping player's confidence up, to increase abilities and build player XP. 
Madden NFL 16   Aug 25, 2015   83   5.7   EA Sports   EA Sports   Sports|Football|Sim|Team|   E
https://www.metacritic.com//game/playstation-4/the-wolf-among-us
Based on the award-winning Fables comic book series by Bill Willingham (published by DC Comics' Vertigo imprint), The Wolf Among Us is a dark, often brutally violent, and mature take on the characters and creatures of fairytale and legend. A five episode series from the creators of the 2012 Game of the Year: The Walking Dead, the p

bsumm
Move or Die   Mar  5, 2019   83   6.2   Those Awesome Guys   Those Awesome Guys   Action|Miscellaneous|General|Party / Minigame|   E10+
https://www.metacritic.com//game/playstation-4/va-11-hall-a-cyberpunk-bartender-action
Great fun, good narrative. The world-building is fantastic and after it finished I wanted to see more of the world. If you're a fan of the Cyberpunk genre, you will have a blast.
VA-11 Hall-A: Cyberpunk Bartender Action   May  2, 2019   83   7.0   Ysbryd Games   Sukeban Games   Simulation|General|   M
https://www.metacritic.com//game/playstation-4/disgaea-4-complete+
The ultimate version of Disgaea 4. It has of cause the main game and all content from “Disgaea 4 A promise revisited” including all additional characters, scenarios, gameplay improvements with some from other Disgaea games and some tweaks to the game.  For this game they add a few things that you might miss but are nice to have like for example skill speed ups or an auto save after battles. For me 

Sayonara Wild Hearts is a dreamy arcade game about riding motorcycles, skateboarding, dance battling, shooting lasers, wielding swords, and breaking hearts at 200 mph. As the heart of a young woman breaks, the balance of the universe is disturbed. A diamond butterfly appears in her dreams and leads her through a highway in the sky, where she finds her other self: the masked biker called The Fool. Journey through a custom-written pop soundtrack, chase scores, and set out to find the harmony of the universe, hidden away in the hearts of Little Death and her star-crossed allies: Dancing Devils, Howling Moons, Stereo Lovers and Hermit 64.
Sayonara Wild Hearts   Sep 19, 2019   82   6.6   Annapurna Interactive   Simogo   Action|Rhythm|Music|   E10+
https://www.metacritic.com//game/playstation-4/watch-dogs-2
Check out the birthplace of the tech revolution as Marcus Holloway, a brilliant young hacker who has fallen victim to ctOS 2.0's predictive algorithms and accused of a crime he did not co

Includes the full 2016 season calendar of 21 tracks, including the brand new Baku circuit in Azerbaijan, and the full roster of 22 drivers and 11 teams, including the new Haas F1 Team. F1 2016 immerses you in not only the on-track excitement of a FORMULA ONE career, including the inclusion of the iconic Safety Car and also the Virtual Safety Car for the first time, but also uniquely offers the drama and vehicle development that goes on behind the scenes. Work with your agent, engineer and team to develop your car in the deepest ever career experience, spanning up to ten seasons. Forge your own path to glory, and rise to be the champion. 
F1 2016   Aug 19, 2016   82   7.9   Codemasters   Codemasters   Racing|Simulation|Automobile|   E
https://www.metacritic.com//game/playstation-4/destiny-2-forsaken
With 2 brand-new destinations to explore, and new weapons, armour and Supers to acquire, Forsaken takes players on an epic journey into the wild frontiers of our solar system where the lines

The progression feels like you're dragging yourself on the ground (not very ninja like), the graphics are mediocre with an 80s uninspiring vibe to it....I don't see what the reviewers liked about this game.
Sairento VR   Jul 10, 2019   82   6.1   Mixed Realms Pte Ltd   Mixed Realms Pte Ltd   Action|Shooter|Light Gun|   M
https://www.metacritic.com//game/playstation-4/huntdown
"He was a man since he was a boy". This IS the best and most entertaining arcade shooter I've ever played! The graphics, music, sound effects
Huntdown   May 12, 2020   82   7.6   Coffee Stain Studios   Coffee Stain Studios   Action|General|Platformer|2D|   T
https://www.metacritic.com//game/playstation-4/galak-z-the-dimensional
Very addictive game, I got hooked fast.  It's a great game with a steep difficulty curve for the first two 'seasons'.  I died many times
Galak-Z: The Dimensional   Aug  4, 2015   82   7.1   17-Bit   17-Bit   General|Shooter|Scrolling|Scrolling|Action|Shooter|Shoot-'Em-Up|Vertical|   E10+
ht